# tokenizer

In [1]:
!pip install wonderwords

In [2]:
import os, json
from tqdm import tqdm

In [3]:
from transformers import LlamaTokenizer
import torch

base_model = "huggyllama/llama-13b"
tokenizer = LlamaTokenizer.from_pretrained(base_model,
                                          )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# gen topic eval dataset

In [4]:
import numpy as np
import random

np.random.seed(42) 

# gen lines eval dataset

In [5]:

def generate_line_index(num_line, idx_opt):
    if idx_opt == "LRT-ABCindex":
        ingredients = ["A", "B", "C", "D", "E", "F"]

        start = 6
        comb = list(itertools.product(ingredients, repeat=start))
        while len(comb) < num_line:
            start += 1
            comb = list(itertools.product(ingredients, repeat=start))
        
        comb = ["".join(i) for i in comb]

        return comb[:num_line]
    elif idx_opt == "LRT-UUID":
        comb = []
        for i in range(num_line):
            comb.append(str(uuid.uuid4()))
        
        return comb
    elif idx_opt == "LRT-NL":
        import wonderwords

        w = wonderwords.RandomWord()
        adjs = w.random_words(num_line, include_categories=["noun"])
        nouns = w.random_words(num_line, include_categories=["noun"])

        comb = []
        for i, (adj, noun) in enumerate(zip(adjs, nouns)):
            comb.append(f"{adj}-{noun}")
        
        return comb
    
def retrieve_expected(lines, random_line_pos):
    correct_line = lines[random_line_pos]
    expected_number = re.search("<\d+>", correct_line)
    if expected_number is not None:
        expected_number = int(expected_number.group()[1:-1])
    else:
        print(f"Got unparsable line: {correct_line}")

    return expected_number, correct_line

def generate_prompt_from_lines(lines):
    prompt = ""
    for l in lines:
        prompt += l
    
    return prompt

In [6]:
import random, re

RECORD_COUNT = 20

ROWS = [4000]
output_dir = "."

avg_len = 0

for n in ROWS:
    output_path = os.path.join(output_dir, f"{n}_lines_en.jsonl")
    f = open(output_path, "w", encoding="utf-8")

    for i in tqdm(list(range(RECORD_COUNT))):          
        prompt_header = "Below is a record of lines I want you to remember. " + \
                        "Each line begins with 'line <line index>' and contains " + \
                        "a '<REGISTER_CONTENT>' at the end of the line as a numerical value. " + \
                        "For each line index, memorize its corresponding <REGISTER_CONTENT>. At " + \
                        "the end of the record, I will ask you to retrieve the corresponding " + \
                        "<REGISTER_CONTENT> of a certain line index. Now the record start:\n\n"
        lines = []

        line_idx_opt = "LRT-NL"

        if line_idx_opt == "LRT":
            line_idxes = list(range(1, n + 1))
            lines.extend([f"line {i}: REGISTER_CONTENT is <{random.randint(1, 50000)}>\n" for i in line_idxes])
            random_idx = random.randint(1, n)
            random_num = random_idx - 1
        else:
            line_idxes = generate_line_index(n, line_idx_opt)
            lines.extend([f"line {i}: REGISTER_CONTENT is <{random.randint(1, 50000)}>\n" for i in line_idxes])
            random_num = random.randint(0, len(line_idxes)-1)
            random_idx = line_idxes[random_num]

        expected_number, correct_line = retrieve_expected(lines, random_num)
        lines.insert(0, f"{prompt_header}")
        prompt_postfix = f"\nNow the record is over. Tell me what is the <REGISTER_CONTENT> in line {random_idx}? I need the number."

        prompt = generate_prompt_from_lines(lines)

        prompt_len = len(tokenizer.encode(prompt))


        avg_len += prompt_len/500

        
        output = {
            "random_idx": (random_idx, random_num), # this is the line to retrieve
            "expected_number": expected_number,
            "num_lines": n,
            "prompt_len":prompt_len,
            "correct_line": correct_line,
            "prompt_postfix": prompt_postfix,
            "prompt": prompt}

        json.dump(output, f, ensure_ascii=False)
        f.write("\n")
    f.close()


avg_len

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.18it/s]


3687.8080000000004

In [ ]:
!head -n 1 {n}_lines_en.jsonl

In [8]:
!wc -l {n}_lines_en.jsonl

      20 4000_lines_en.jsonl
